In [38]:
import utente
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100

In [2]:
# Carregamento estrutura dados (passar parâmetro forceFetch = True para forçar leitura BD >> Demorado!)
listaUtentes = utente.ListaUtentes(forceFetch = False, sqlLimit = 99999999999) # Carrega dados locais a partir de PICKLE

Loading from ./temp/ListaUtentes.pickle
Finished


In [3]:
# Criação do dataframe pandas
df = pd.DataFrame(listaUtentes.generateDataset())
listaUtentes = None # Limpa listaUtentes depois de gerar Dataset

## Limpeza das variáveis

Substituição de `None` por 0's e correcção do Types

In [10]:
# DataNascimento
df['DataNascimento'].fillna(value=np.nan, inplace = True)
df['DataNascimento'] = pd.to_datetime(df['DataNascimento'])

In [28]:
# Substitui por 0 todos Nones nas colunas que 
# começam por NrAnulacoesPorMotivo ou NrApresentacoesComResultado
filter_col = [col for col in df if col.startswith('NrAnulacoesPorMotivo') 
                                   or col.startswith('NrApresentacoesComResultado')]

for col in filter_col:
    df[col].fillna(value=0, inplace = True);
    df[col]=df[col].astype('int');

In [29]:
# Empregado?
df['Empregado'] = df['Empregado'].astype('category')

## Preparação dos dados

In [88]:
dfM = df[df.Empregado=='S']

In [89]:
# None -> NaN
dfM['CursoAnoConclusao'].fillna(value=np.nan, inplace = True)
dfM['NrDescendentesCargo'].fillna(value=np.nan, inplace = True)


/Users/helioteixeira/Documents/Code/Python/IEFP_DSSG/venv/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [90]:
colNamesToDummify = ['Sexo', 
                     'RSI', 
                     'CategoriaAtrib', 
                     'Empregado', 
                     'EstadoCivil', 
                     'FormacaoProf', 
                     'HabilitacaoEscolar',
                     'Nacionalidade',
                     'Qualificacao']
for colName in colNamesToDummify:
    df_dummies = pd.get_dummies(dfM[colName], prefix=colName+'_')
    dfM = pd.concat([dfM, df_dummies], axis = 1)
    dfM.drop(colName, axis = 1)
    
colNamesToDrop = ['AreaCurso', 
                  'AreaFormacao', 
                  'CnpAnterior', 
                  'ConjugeMotivoIndisponibilidade',
                  'CursoEstabelecimentoEnsinoSup',
                  'Data',
                  'DataNascimento',
                  'LTU',
                  'Utente']

dfM.drop(colNamesToDummify, axis = 1, inplace = True)
dfM.drop(colNamesToDrop, axis = 1, inplace = True)

In [91]:
dfM.describe()

,DiasDesemprego,Idade,NivelDeficiencia,NrAnulacoes,NrAnulacoesPorMotivo_CADUCIDADE DO DOCUMENTO DE IDENTIFICAÇÃO,NrAnulacoesPorMotivo_COLOCAÇÃO - CANDIDATURA EXTERNA,"NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA DE OUTREM","NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA PRÓPRIA",NrAnulacoesPorMotivo_CORRECÇÃO COLOCAÇÃO FME,NrAnulacoesPorMotivo_CORRECÇÃO REINSCRIÇÃO FME,NrAnulacoesPorMotivo_CRIAÇÃO DO PRÓPRIO EMPREGO,NrAnulacoesPorMotivo_CUMPRIMENTO SERVIÇO MILITAR,NrAnulacoesPorMotivo_DESISTIU,NrAnulacoesPorMotivo_DESISTÊNCIA OU EXCLUSÃO DE FORMAÇÃO,NrAnulacoesPorMotivo_DESISTÊNCIA OU EXCLUSÃO DE MEDIDAS DE EMPREGO (EXCLUI TRABALHO SOCIALMENTE NECESSÁRIO),NrAnulacoesPorMotivo_DESISTÊNCIA OU EXCLUSÃO DE TRABALHO SOCIALMENTE NECESSÁRIO,NrAnulacoesPorMotivo_DETENÇÃO EM ESTABELECIMENTO PRISIONAL,"NrAnulacoesPorMotivo_EMIGRAÇÃO, AUSÊNCIA DO PAÍS",NrAnulacoesPorMotivo_FALECIMENTO,NrAnulacoesPorMotivo_FALTA A CONVOCATÓRIA,NrAnulacoesPorMotivo_FALTA AO CONTROLO,NrAnulacoesPorMotivo_FALTA À APRESENTAÇÃO,NrAnulacoesPorMotivo_FREQUÊNCIA DE CURSO DE FORMAÇÃO PROFISSIONAL DO IEFP,NrAnulacoesPorMotivo_FREQUÊNCIA DE ESTÁGIO PROFISSIONAL PEPAC,NrAnulacoesPorMotivo_FREQUÊNCIA DE ESTÁGIO PROFISSIONAL PEPAL,NrAnulacoesPorMotivo_FREQUÊNCIA DE ESTÁGIO PROFISSIONAL PEPAP,NrAnulacoesPorMotivo_FREQUÊNCIA DE FORMAÇÃO DO IEFP (ENTRADA SGFOR),NrAnulacoesPorMotivo_FREQUÊNCIA DE FORMAÇÃO EM ENTIDADE EXTERNA AO IEFP,NrAnulacoesPorMotivo_INCAPACIDADE PROLONGADA / PERMANENTE PARA O TRABALHO,NrAnulacoesPorMotivo_INCUMPRIMENTO DA APRESENTAÇÃO QUINZENAL,NrAnulacoesPorMotivo_INSCRIÇÃO CANDIDATURA EXTERNA,NrAnulacoesPorMotivo_INSERÇÃO NOUTROS PROGRAMAS/MEDIDAS EMPREGO (SIEF),"NrAnulacoesPorMotivo_INSERÇÃO POR CONTA OUTRÉM, NA SEQUÊNCIA DE PROGRAMA","NrAnulacoesPorMotivo_INÍCIO DE LICENÇA DE MATERNIDADE, PATERNIDADE OU ADOPÇÃO",NrAnulacoesPorMotivo_OUTROS MOTIVOS,NrAnulacoesPorMotivo_RECUSA DE EMPREGO CONVENIENTE,NrAnulacoesPorMotivo_RECUSA DE FORMALIZAÇÃO DE PPE,NrAnulacoesPorMotivo_RECUSA DE FORMAÇÃO PROFISSIONAL,NrAnulacoesPorMotivo_RECUSA DE INTERVENÇÃO,NrAnulacoesPorMotivo_RECUSA DE TRABALHO SOCIALMENTE NECESSÁRIO,NrAnulacoesPorMotivo_REFORMOU-SE,NrAnulacoesPorMotivo_REINÍCIO DE ESTUDOS,NrAnulacoesPorMotivo_TRANSFERÊNCIA DE CENTRO DE EMPREGO,NrAnulacoesPorMotivo_TRANSFERÊNCIA REGIÕES AUTÓNOMAS,NrApresentacoes,NrApresentacoesComResultado_CADUCIDADE DO DOCUMENTO DE IDENTIFICAÇÃO,NrApresentacoesComResultado_COLOCAÇÃO - CANDIDATURA EXTERNA,"NrApresentacoesComResultado_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA DE OUTREM","NrApresentacoesComResultado_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA PRÓPRIA",NrApresentacoesComResultado_CORRECÇÃO COLOCAÇÃO FME,...,Nacionalidade__GÂMBIA,Nacionalidade__HOLANDA,Nacionalidade__HUNGRIA,Nacionalidade__INDONÉSIA,Nacionalidade__IRLANDA,Nacionalidade__IRÃO,Nacionalidade__ITÁLIA,Nacionalidade__JUGOSLÁVIA,Nacionalidade__LETÓNIA,Nacionalidade__LUXEMBURGO,Nacionalidade__MALI,Nacionalidade__MARROCOS,Nacionalidade__MOLDÁVIA,Nacionalidade__MOÇAMBIQUE,Nacionalidade__MÉXICO,Nacionalidade__NEPAL,Nacionalidade__OUTRO,Nacionalidade__PANAMÁ,Nacionalidade__PAQUISTÃO,Nacionalidade__POLÓNIA,Nacionalidade__PORTUGAL,Nacionalidade__QUIRGUIZISTÃO,Nacionalidade__REINO UNIDO,Nacionalidade__REPÚBLICA CHECA,Nacionalidade__ROMÉNIA,Nacionalidade__RÚSSIA,Nacionalidade__SENEGAL,Nacionalidade__SUIÇA,Nacionalidade__SUÉCIA,Nacionalidade__SÃO TOMÉ E PRINCIPE,Nacionalidade__SÉRVIA,Nacionalidade__TUNÍSIA,Nacionalidade__TURQUIA,Nacionalidade__UCRÂNIA,Nacionalidade__URUGUAI,Nacionalidade__UZBEQUISTÃO,Nacionalidade__VENEZUELA,Nacionalidade__ZIMBABWE,Nacionalidade__ÁFRICA DO SUL,Nacionalidade__ÁUSTRIA,Nacionalidade__ÍNDIA,Qualificacao__0,Qualificacao__1,Qualificacao__2,Qualificacao__3,Qualificacao__4,Qualificacao__5,Qualificacao__6,Qualificacao__7,Qualificacao__8
count,23215.000000,23215.000000,23215.000000,23215.000000,23215.000000,23215.000000,23215.000000,23215.000000,23215.000000,23215.0,23215.000000,23215.0,23215.000000,23215.000000,23215.

## Modelo LASSO

In [92]:
from sklearn.model_selection import train_test_split
#Split data in train set and test set

y = dfM['DiasDesemprego']
X = dfM.drop('DiasDesemprego', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# #############################################################################
# Lasso
from sklearn.linear_model import Lasso

alpha = 0.1
lasso = Lasso(alpha=alpha)

y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
r2_score_lasso = r2_score(y_test, y_pred_lasso)
print(lasso)
print("r^2 on test data : %f" % r2_score_lasso)

ValueError: could not convert string to float: '0,0'